In [2]:
import xgboost
from xgboost import XGBClassifier
import matplotlib.pylab as pl
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import os
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import pickle
from sklearn.neural_network import MLPClassifier

In [4]:
path = '/Users/kj/Library/CloudStorage/OneDrive-한림대학교/연구 및 과제/박사논문/30'

if not path.endswith('/'):
    path += '/'
if not os.path.isdir(path):
    os.mkdir(path)

In [5]:
X = pd.read_excel('/Users/kj/Library/CloudStorage/OneDrive-한림대학교/연구 및 과제/박사논문/reafter30.xlsx')
y = X['30_days_label']

drop_list = ['30_days_label']
X = X.drop(drop_list, axis=1)

fea_list = pd.read_excel('/Users/kj/Library/CloudStorage/OneDrive-한림대학교/연구 및 과제/박사논문/feature_list_reafter30.xlsx')
nominal_fea = fea_list[fea_list['Nominal']==1]['Type_Short_Name'].tolist()
nominal_fea = list(set(nominal_fea) & set(X.columns))
X = pd.get_dummies(X, columns=nominal_fea, drop_first=True)

print(X.columns)

Index(['MIC: IMP or MPM', 'WBC, Low', 'PLT, High', 'MPV', 'PT', 'aPTT', 'ALP',
       'Total Cheloesterol', 'K, Low', 'cl, Low', 'protein',
       'colonization_sputum_1'],
      dtype='object')


In [6]:
print(X.columns)
print(X.shape)
print('# samples: ', X.shape[0])
print('# positive samples: ', sum(y==1))
print('# negative samples: ', sum(y==0))
print('# features: ', X.shape[1])

Index(['MIC: IMP or MPM', 'WBC, Low', 'PLT, High', 'MPV', 'PT', 'aPTT', 'ALP',
       'Total Cheloesterol', 'K, Low', 'cl, Low', 'protein',
       'colonization_sputum_1'],
      dtype='object')
(639, 12)
# samples:  639
# positive samples:  72
# negative samples:  567
# features:  12


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

####XGBOOST

In [8]:
# XGBOOST하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'learning_rate': [0.02, 0.03, 0.04, 0.05, 0.06 ],
    'max_depth': [1, 2, 3, 4, 5, 6, 7, 8],
    'subsample': [0.5, 0.6, 0.7, 0.8],
}

In [9]:
# XGBoost 분류기 초기화
xlf = xgboost.XGBClassifier(objective='binary:logistic', random_state=42)

In [10]:

grid_search = GridSearchCV(estimator=xlf, param_grid=param_grid, scoring='roc_auc', cv=5, verbose=2, n_jobs=-1)

In [11]:

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 800 candidates, totalling 4000 fits
[CV] END learning_rate=0.02, max_depth=1, n_estimators=100, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=300, subsample=0.5; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=300, subsample=0.7; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=400, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=400, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=500, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=500, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=2, n_estimators=300, subsample=0.5; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=2, n_estimators=300, subsample=0.5; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=2, n_estimators=300, subsample=0.5; total time=   0

[CV] END learning_rate=0.02, max_depth=1, n_estimators=100, subsample=0.5; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=200, subsample=0.5; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=200, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=200, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=300, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=300, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=500, subsample=0.5; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=500, subsample=0.5; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=2, n_estimators=100, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=2, n_estimators=100, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=2, n_estimators=200, 

[CV] END learning_rate=0.02, max_depth=1, n_estimators=100, subsample=0.7; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=300, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=400, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=400, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=2, n_estimators=100, subsample=0.5; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=2, n_estimators=100, subsample=0.5; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=2, n_estimators=100, subsample=0.7; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=2, n_estimators=100, subsample=0.7; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=2, n_estimators=100, subsample=0.7; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=2, n_estimators=100, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=2, n_estimators=300, 

[CV] END learning_rate=0.02, max_depth=1, n_estimators=100, subsample=0.5; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=200, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=200, subsample=0.7; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=300, subsample=0.5; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=300, subsample=0.7; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=500, subsample=0.5; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=500, subsample=0.5; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=2, n_estimators=100, subsample=0.5; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=2, n_estimators=100, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=2, n_estimators=100, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=2, n_estimators=100, 

[CV] END learning_rate=0.02, max_depth=1, n_estimators=100, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=100, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=100, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=200, subsample=0.5; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=200, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=200, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=300, subsample=0.5; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=300, subsample=0.7; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=400, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=400, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=500, 

[CV] END learning_rate=0.02, max_depth=3, n_estimators=100, subsample=0.7; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=3, n_estimators=100, subsample=0.7; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=4, n_estimators=100, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=4, n_estimators=100, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=4, n_estimators=100, subsample=0.7; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=4, n_estimators=100, 

[CV] END learning_rate=0.02, max_depth=1, n_estimators=100, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=300, subsample=0.7; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=400, subsample=0.7; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=400, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=500, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=1, n_estimators=500, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=2, n_estimators=200, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=2, n_estimators=200, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=2, n_estimators=200, subsample=0.7; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=2, n_estimators=200, subsample=0.7; total time=   0.0s
[CV] END learning_rate=0.02, max_depth=2, n_estimators=500, 

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.02, 0.03, 0.04, 0.05, 0.06],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8],
                         'n_estimators': [100, 200, 300, 400, 500],
                         'subsample': [0.5, 0.6, 0.7, 0.8]},
             scoring='roc_auc', verbose=2)

In [12]:
# 최적 하이퍼파라미터 출력
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'learning_rate': 0.04, 'max_depth': 2, 'n_estimators': 200, 'subsample': 0.5}


In [13]:

best_params = grid_search.best_params_

model = XGBClassifier(**best_params)

model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)[:, 1]

auc_score = roc_auc_score(y_test, y_pred_proba)
print(f'Test AUC: {auc_score:.4f}')

Test AUC: 0.8641
[CV] END learning_rate=0.04, max_depth=5, n_estimators=400, subsample=0.7; total time=   0.1s
[CV] END learning_rate=0.04, max_depth=5, n_estimators=400, subsample=0.7; total time=   0.1s
[CV] END learning_rate=0.04, max_depth=5, n_estimators=400, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.04, max_depth=7, n_estimators=100, subsample=0.5; total time=   0.0s
[CV] END learning_rate=0.04, max_depth=7, n_estimators=100, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.04, max_depth=7, n_estimators=100, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.04, max_depth=7, n_estimators=100, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.04, max_depth=7, n_estimators=100, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.04, max_depth=7, n_estimators=100, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.04, max_depth=7, n_estimators=100, subsample=0.7; total time=   0.0s
[CV] END learning_rate=0.04, max_depth=7, n

[CV] END learning_rate=0.05, max_depth=2, n_estimators=200, subsample=0.7; total time=   0.0s
[CV] END learning_rate=0.05, max_depth=2, n_estimators=200, subsample=0.7; total time=   0.0s
[CV] END learning_rate=0.05, max_depth=2, n_estimators=200, subsample=0.7; total time=   0.0s
[CV] END learning_rate=0.05, max_depth=2, n_estimators=200, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.05, max_depth=2, n_estimators=200, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.05, max_depth=2, n_estimators=200, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.05, max_depth=2, n_estimators=200, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.05, max_depth=2, n_estimators=200, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.05, max_depth=3, n_estimators=200, subsample=0.5; total time=   0.0s
[CV] END learning_rate=0.05, max_depth=3, n_estimators=200, subsample=0.5; total time=   0.0s
[CV] END learning_rate=0.05, max_depth=3, n_estimators=200, 

[CV] END learning_rate=0.04, max_depth=4, n_estimators=400, subsample=0.6; total time=   0.1s
[CV] END learning_rate=0.04, max_depth=4, n_estimators=400, subsample=0.6; total time=   0.0s
[CV] END learning_rate=0.04, max_depth=4, n_estimators=400, subsample=0.6; total time=   0.1s
[CV] END learning_rate=0.04, max_depth=4, n_estimators=400, subsample=0.6; total time=   0.1s
[CV] END learning_rate=0.04, max_depth=4, n_estimators=400, subsample=0.7; total time=   0.0s
[CV] END learning_rate=0.04, max_depth=4, n_estimators=400, subsample=0.7; total time=   0.1s
[CV] END learning_rate=0.04, max_depth=5, n_estimators=400, subsample=0.8; total time=   0.1s
[CV] END learning_rate=0.04, max_depth=5, n_estimators=400, subsample=0.8; total time=   0.1s
[CV] END learning_rate=0.04, max_depth=5, n_estimators=400, subsample=0.8; total time=   0.1s
[CV] END learning_rate=0.04, max_depth=5, n_estimators=400, subsample=0.8; total time=   0.1s
[CV] END learning_rate=0.04, max_depth=5, n_estimators=500, 

[CV] END learning_rate=0.04, max_depth=7, n_estimators=500, subsample=0.6; total time=   0.1s
[CV] END learning_rate=0.04, max_depth=7, n_estimators=500, subsample=0.6; total time=   0.1s
[CV] END learning_rate=0.04, max_depth=7, n_estimators=500, subsample=0.6; total time=   0.1s
[CV] END learning_rate=0.04, max_depth=7, n_estimators=500, subsample=0.6; total time=   0.1s
[CV] END learning_rate=0.04, max_depth=7, n_estimators=500, subsample=0.7; total time=   0.1s
[CV] END learning_rate=0.04, max_depth=7, n_estimators=500, subsample=0.7; total time=   0.1s
[CV] END learning_rate=0.05, max_depth=1, n_estimators=100, subsample=0.7; total time=   0.0s
[CV] END learning_rate=0.05, max_depth=1, n_estimators=100, subsample=0.7; total time=   0.0s
[CV] END learning_rate=0.05, max_depth=1, n_estimators=100, subsample=0.7; total time=   0.0s
[CV] END learning_rate=0.05, max_depth=1, n_estimators=100, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.05, max_depth=1, n_estimators=100, 

In [17]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import make_scorer, average_precision_score


dnn_params = {
    'hidden_layer_sizes': [(100,), (100, 100), (50, 50, 50)], # 은닉층 크기 및 층 수 시도
    'max_iter': [1000], # 최대 반복 횟수
    'learning_rate_init': [0.001, 0.01], # 학습률 초기값
    'alpha': [0.0001, 0.001, 0.01], # L2 규제 항
    'batch_size': [64, 128], # 배치 크기
    'solver': ['adam', 'lbfgs'], # 솔버
}


auprc_scorer = make_scorer(average_precision_score, needs_proba=True)


dnn_model = MLPClassifier(random_state=7)
grid_search = GridSearchCV(estimator=dnn_model, param_grid=dnn_params, scoring=auprc_scorer, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train) # X_train, y_train은 훈련 데이터와 타겟 변수


print("Best parameters found: ", grid_search.best_params_)


best_dnn_model = grid_search.best_estimator_


cv_scores = cross_val_score(best_dnn_model, X_train, y_train, cv=5, scoring=auprc_scorer)
print("AUPRC Cross-validation scores: ", cv_scores)


/Users/kj/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/kj/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/kj/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH

Best parameters found:  {'alpha': 0.0001, 'batch_size': 128, 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.01, 'max_iter': 1000, 'solver': 'adam'}
AUPRC Cross-validation scores:  [0.46813071 0.65236483 0.68789255 0.44572209 0.51208949]


#### Logistic Regression

In [18]:
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10],
    'penalty': ['l1', 'l2']
}

In [19]:
# Logistic Regression 모델 초기화
log_reg = LogisticRegression(max_iter=10000, random_state=42)

In [20]:
# GridSearchCV 초기화
grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, scoring='roc_auc', cv=5, verbose=2, n_jobs=-1)

In [21]:
# 훈련 데이터에 대해 그리드 서치 수행
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/Users/kj/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/kj/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
25 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the follow

GridSearchCV(cv=5,
             estimator=LogisticRegression(max_iter=10000, random_state=42),
             n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10],
                         'penalty': ['l1', 'l2']},
             scoring='roc_auc', verbose=2)

In [22]:
# 최적 하이퍼파라미터 출력
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'C': 0.001, 'penalty': 'l2'}


In [23]:
# grid_search.best_params_에서 얻은 최적의 하이퍼파라미터를 사용
best_params = grid_search.best_params_

# LogisticRegression 클래스를 사용하여 새 모델 인스턴스 생성 및 최적의 하이퍼파라미터 적용
model = LogisticRegression(**best_params)

# 최적의 하이퍼파라미터로 모델 훈련
model.fit(X_train, y_train)

# 훈련된 모델로 테스트 데이터의 예측 확률을 계산
y_pred_proba = model.predict_proba(X_test)[:, 1]

# AUC 계산
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f'Test AUC: {auc_score:.4f}')

Test AUC: 0.8383


/Users/kj/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END learning_rate=0.002, max_depth=3, n_estimators=100, subsample=0.4; total time=   0.1s
[CV] END learning_rate=0.002, max_depth=3, n_estimators=500, subsample=0.2; total time=   0.2s
[CV] END learning_rate=0.002, max_depth=3, n_estimators=500, subsample=0.8; total time=   0.3s
[CV] END learning_rate=0.002, max_depth=3, n_estimators=1000, subsample=0.4; total time=   0.5s
[CV] END learning_rate=0.002, max_depth=5, n_estimators=100, subsample=0.2; total time=   0.1s
[CV] END learning_rate=0.002, max_depth=5, n_estimators=100, subsample=0.2; total time=   0.1s
[CV] END learning_rate=0.002, max_depth=5, n_estimators=100, subsample=0.4; total time=   0.1s
[CV] END learning_rate=0.002, max_depth=5, n_estimators=100, subsample=0.8; total time=   0.1s
[CV] END learning_rate=0.002, max_depth=5, n_estimators=500, subsample=0.2; total time=   0.3s
[CV] END learning_rate=0.002, max_depth=5, n_estimators=500, subsample=0.6; total time=   0.4s
[CV] END learning_rate=0.002, max_depth=5, n_esti